In [3]:
import numpy as np
import pandas as pd

In [2]:
# задание 
# построить матрицу наблюдений  со сгенерированными полиномиальными признаками
# полиномиальная регрессия второй степени с тремя факторами
# сперва составим обычную матрицу, расположив векторы (факторы) в столбцах
A = np.array([
    [1, 3, -2, 1, 5, 13, 1],
    [3, 4, 5, -2, 4, 11, 3],
    [4, 5, 2, 2, 6, 8, -1],
]).T
print(A)

[[ 1  3  4]
 [ 3  4  5]
 [-2  5  2]
 [ 1 -2  2]
 [ 5  4  6]
 [13 11  8]
 [ 1  3 -1]]


In [4]:
from sklearn.preprocessing import PolynomialFeatures
# указываем степень полинома =2, а также то, что нам нужна генерация столбца из 1
poly = PolynomialFeatures(degree=2, include_bias=True)
A_poly = poly.fit_transform(A)
display(pd.DataFrame(A_poly))


,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,3.0,4.0,1.0,3.0,4.0,9.0,12.0,16.0
1,1.0,3.0,4.0,5.0,9.0,12.0,15.0,16.0,20.0,25.0
2,1.0,-2.0,5.0,2.0,4.0,-10.0,-4.0,25.0,10.0,4.0
3,1.0,1.0,-2.0,2.0,1.0,-2.0,2.0,4.0,-4.0,4.0
4,1.0,5.0,4.0,6.0,25.0,20.0,30.0,16.0,24.0,36.0
5,1.0,13.0,11.0,8.0,169.0,143.0,104.0,121.0,88.0,64.0
6,1.0,1.0,3.0,-1.0,1.0,3.0,-1.0,9.0,-3.0,1.0


столбец 0 — единичный, он отвечает за слагаемое с нулевой степенью полинома (любое число в степени 0 даёт единицу).

столбцы 1,2 и 3 — это наши исходные признаки (векторы х1, х2 и х3).

столбцы 4, 5 и 6 — произведения первого столбца со всеми столбцами: х1х1 = x1^2, x1x2 и x1x3 соответственно.

столбцы 7 и 8 — произведения второго столбца со столбцами 2 и 3: x2x2=x2^2 и x2x3.

столбец 9 - произведение третьего столбца с самим собой: x3x3=x3^2.

In [5]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
boston_data = pd.read_csv('housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [6]:
# чтобы не дублировать код, напишем функцию
# она принимает на вход матрицу наблюдений, вектор ответов и степень полинома 
# а возвращает матрицу с полиномиальными признаками, вектор предсказаний и коэффециенты регрессии, найденные по МНК  
def polynomial_regression(X, y, k):
    poly = PolynomialFeatures(degree=k, include_bias=True)
    X_poly = poly.fit_transform(X)
    w_hat = np.linalg.inv(X_poly.T@X_poly)@X_poly.T@y
    y_pred = X_poly@w_hat
    return X_poly, y_pred, w_hat

In [8]:
# выделяем интересующие нас признаки и строим полиномы (от 1 до 5 степени включительно)
A = boston_data[['LSTAT', 'PTRATIO', 'RM', 'CRIM']]
y = boston_data[['PRICE']]
 
A_poly, y_pred, w_hat = polynomial_regression(A, y, 1)
A_poly2, y_pred2, w_hat2 = polynomial_regression(A, y, 2)
A_poly3, y_pred3, w_hat3 = polynomial_regression(A, y, 3)
A_poly4, y_pred4, w_hat4 = polynomial_regression(A, y, 4)
A_poly5, y_pred5, w_hat5 = polynomial_regression(A, y, 5)

In [9]:
# посмотрим на качесто построенных регрессий, вычислив метрику
from sklearn.metrics import mean_absolute_percentage_error
# сравним результаты по значению средней абсолютной процентной ошибки (MAPE)
print('MAPE для полинома 1-й степени {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred)*100))
print('MAPE для полинома 2-й степени  {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred2)*100))
print('MAPE для полинома 3-й степени  {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred3)*100))
print('MAPE для полинома 4-й степени  {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred4)*100))
print('MAPE для полинома 5-й степени  {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred5)*100))

MAPE для полинома 1-й степени 18.20%
MAPE для полинома 2-й степени  13.41%
MAPE для полинома 3-й степени  12.93%
MAPE для полинома 4-й степени  10.75%
MAPE для полинома 5-й степени  346.97%


Проанализировав МАРЕ, можем сделать вывод, что модель вообще не описывает зависимость в исходных данных — её прогноз не имеет никакого отношения к действительности.



In [13]:
# анализируем 
display(pd.DataFrame(w_hat5).describe())

,PRICE
count,1.260000e+02
mean,5.144793e+02
std,1.786794e+04
min,-1.082477e+05
25%,-8.773205e-01
50%,8.713042e-07
75%,1.204568e+00
max,1.654534e+05


In [14]:
# взглянем на корреляционную матрицу для факторов, на которых мы строим полином пятой степени
# считаем матрицу корреляций (без столбца из единиц)
# Для удобства расчёта матрицы корреляций обернём матрицу  в DataFrame и воспользуемся методом corr()
C = pd.DataFrame(A_poly5[:,1:]).corr()
# считаем ранг корреляционной матрицы
print('Ранг корреляционной матрицы:', np.linalg.matrix_rank(C))
# считаем кол-во факторов (не включая столбец из единиц)
print('Количество факторов:', A_poly5[:,1:].shape[1])



Ранг корреляционной матрицы: 110
Количество факторов: 125


Ранг корреляционной матрицы не максимален!

Это значит, что в корреляционной матрице присутствуют единичные корреляции, а в исходной матрице — линейно зависимые столбцы.

В процессе перемножения каких-то из столбцов при создании полинома пятой степени получился такой полиномиальный фактор, который линейно выражается через другие факторы.

В результате при вычислении обратной матрицы  у нас получилось деление на число, близкое к 0, а элементы обратной матрицы получились просто огромными. Отсюда и появились явно неверные степени коэффициентов, которые дают далёкий от действительности прогноз, что приводит к отрицательной метрике.

In [16]:
# построим все ту же полиномиальную регрессию, но уже с помощью sklearn
# плюс посмотрим стандартное отклонение по коэффициентам регрессии
from sklearn.linear_model import LinearRegression
def polynomial_regression_sk(X, y, k):
    poly = PolynomialFeatures(degree=k, include_bias=False)
    X_poly = poly.fit_transform(X)
    lr = LinearRegression().fit(X_poly, y)
    y_pred = lr.predict(X_poly)
    return X_poly, y_pred, lr.coef_

A = boston_data[['LSTAT', 'PTRATIO', 'RM', 'CRIM']]
y = boston_data[['PRICE']]

for k in range(1, 6):
    A_poly, y_pred, w_hat = polynomial_regression_sk(A, y, k)
    print(
        "MAPE для полинома степени {} — {:.2f}%, СКО — {:.0f}".format(
            k, mean_absolute_percentage_error(y, y_pred)*100, w_hat.std()
        )

    )
    # а тут уже построение полинома пятой степени прошло успешно!

MAPE для полинома степени 1 — 18.20%, СКО — 2
MAPE для полинома степени 2 — 13.41%, СКО — 5
MAPE для полинома степени 3 — 12.93%, СКО — 9
MAPE для полинома степени 4 — 10.74%, СКО — 304
MAPE для полинома степени 5 — 9.02%, СКО — 17055


In [17]:
print(10.4 + 8*1 + 0.5*4 + 3*1 + 0.4*16 + 0)

29.799999999999997


In [19]:
# найти коэффициенты полиномиальной регрессии
B = np.array([
    [1, 1, 1, 1], 
    [1, 3, -2, 9], 
    [1, 9, 4, 81]
]).T
y = np.array([3, 7, -5, 21]).T
w_hat = np.linalg.inv(B.T@B)@B.T@y
w_hat

array([ 0.11446013,  2.46095638, -0.01608801])